## HW10

+ just showing how to compute matrix bigA and the covariance matrix of bigX
* there are probably other and better ways to do the same
+ the rest shoud be straightforward

In [1]:
from statsmodels.tsa.tsatools import unvec, vec
import numpy as np

In [2]:
def get_SSmat(d, phi1, phi2, sigma_eta, sigma_eps):
    """
    State Space matrices
    """
    
    C = np.array([[1., -1.]])
    d = np.array([[d]])
    A = np.array([[phi1, phi2],[1., 0]])
    selection = np.array([[1.],[0]])
    state_cov = np.array([[sigma_eps ** 2]])
    SigE = selection@state_cov@selection.T
    SigV = np.array([[sigma_eta ** 2]])
    
    return d, A, C, SigE, SigV

def get_SigX0(A, SigE):
    """
    Covariance matrix of the initial state
    """
    vecSigX0=np.linalg.solve(np.eye(vec(SigE).shape[0]) - np.kron(A, A), vec(SigE))
    SigX0 = unvec(vecSigX0)
    
    return SigX0

def get_SigBigE(A, SigE, T):
    """
    A function to compute the covariance matrix of bigE. Its a block diagonal matrix with the cov matrix of X0 in the 
    first block and then the identical covariance matrices of the state innovations
    """
    
    n_states = A.shape[0]
    assert n_states == SigE.shape[0]
    
    SigX0 = get_SigX0(A, SigE)
    
    SigBigE = np.zeros(((T+1)*n_states, (T+1)*n_states))
    SigBigE[:n_states, :n_states] = SigX0
    SigBigE[n_states:, n_states:] = np.kron(np.eye(T), SigE)
    
    return SigBigE
    

def get_bigA(A, T):
    """
    X = bigA \times E
    """
    
    n_states = A.shape[0]
    
    bigA = np.zeros((T*n_states, (T+1)*n_states))
    bigA[:n_states, :n_states] = A
    bigA[:n_states, n_states:n_states*2] = np.eye(n_states)

    for i in range(2, T+1):
        last_row_indices = range(n_states*(i-2), n_states*(i-1))
        row_indices = range(n_states*(i-1),n_states*i)
        bigA[row_indices, :n_states] = A@bigA[last_row_indices, :n_states]
        bigA[row_indices, n_states:] = bigA[last_row_indices, :-n_states]
    
    return bigA

def get_sample_X(A, SigE, T):
    """
    A function to simulate data from the model for X
    """
    
    n_states = A.shape[0]
    
    SigX0 = get_SigX0(A, SigE)
    x0 = np.random.multivariate_normal(np.zeros(n_states), SigX0)
    e_1 = np.random.multivariate_normal(np.zeros(n_states), SigE)
    
    x_1 = A@x0 + e_1

    X = x_1
    E = np.hstack((x0, e_1))
    x_t = x_1

    for i in range(2, T+1):
        e_t = np.random.multivariate_normal(np.zeros(2), SigE)
        x_t = A@x_t + e_t

        X = np.hstack((X, x_t))
        E = np.hstack((E, e_t))
        
    return X, E

Get State space matrices

In [3]:
d, A, C, SigE, SigV = get_SSmat(d=0.8119, phi1 = 1.5303, phi2 = -0.6097, sigma_eta = 0.6893, sigma_eps =0.6199)

Get bigA matrix for some T

In [4]:
T = 1000
bigA = get_bigA(A, T)

Simulate data for X and E using the recursive representation

In [5]:
X, E = get_sample_X(A=A, SigE=SigE, T=T)

Given E and bigA, generate another sample X using the bigA matrix

In [6]:
X2 = bigA @ E

Check that the two samples are the same

In [7]:
np.testing.assert_allclose(X, X2)

In [8]:
SigBigE = get_SigBigE(A=A, SigE=SigE, T=T)

In [9]:
SigBigX = bigA @ SigBigE @ bigA.T